In [1]:
from pathlib import Path
import pickle
import numpy as np
import pandas as pd
import os
import re

%load_ext autoreload
%autoreload 2

In [15]:
path_data_dir = Path().cwd().parent.parent / "data"
print(path_data_dir)

/home/tim/Documents/feat-store/data


In [24]:
# load high-level labels
df_labels = pd.read_csv(path_data_dir / "processed/cnc/high_level_dummy.csv")
# df_labels = pd.read_csv(path_data_dir / "processed/cnc/high_level_labels_MASTER_update2020-08-06_new-jan-may-data_with_case.csv")
df_labels.head()

,unix_date,date,cut_dir,part,file_name,tools,len_cut,no_points,signals_names,failed,case_tool_54,failed_tools,comment
0,1540298755,2018-10-23 08:45:00,/home/tim/Documents/Checkfluid-Project/data/ra...,KS-NPC-6FF,Data_KS-NPC-6FF_0.mat,1 3 5 8 11 13 15 17 21 22 23 32 36 37 54 57,177.0,177000,current_main current_sub cut_signal error_x er...,1.0,1,54 15,NaN
1,1540298934,2018-10-23 08:48:00,/home/tim/Documents/Checkfluid-Project/data/ra...,KS-NPC-6FF,Data_KS-NPC-6FF_1.mat,1 3 5 8 11 13 15 17 21 22 23 32 36 37 54 57,177.0,177000,current_main current_sub cut_signal error_x er...,1.0,1,57,NaN
2,1540299114,2018-10-23 08:51:00,/home/tim/Documents/Checkfluid-Project/data/ra...,KS-NPC-6FF,Data_KS-NPC-6FF_2.mat,1 3 5 8 11 13 15 17 21 22 23 32 36 37 54 57,177.0,177000,current_main current_sub cut_signal error_x er...,NaN,2,NaN,NaN
3,1540299293,2018-10-23 08:54:00,/home/tim/Documents/Checkfluid-Project/data/ra...,KS-NPC-6FF,Data_KS-NPC-6FF_3.mat,1 3 5 8 11 13 15 17 21 22 23 32 36 37 54 57,177.0,177000,current_main current_sub cut_signal error_x er...,3.0,3,54 57 15,NaN
4,1540299472,2018-10-23 08:57:00,/home/tim/Documents/Checkfluid-Project/data/ra...,KS-NPC-6FF,Data_KS-NPC-6FF_4.mat,1 3 5 8 11 13 15 17 21 22 23 32 36 37 54 57,177.0,177000,current_main current_sub cut_signal error_x er...,NaN,4,NaN,NaN


In [29]:
# load feature dataframe as df
df = pd.read_csv(path_data_dir / "processed/cnc/cnc_features/cnc_features_54.csv")
df["unix_date"] = df["id"].apply(lambda x: int(x.split("_")[0]))
df["tool_no"] = df["id"].apply(lambda x: int(x.split("_")[-2]))
df["index_no"] = df["id"].apply(lambda x: int(x.split("_")[-1]))
df.head()

,id,current__length,current__kurtosis,unix_date,tool_no,index_no
0,1540298755_54_0,2624.0,-1.802253,1540298755,54,0
1,1540298755_54_1,8504.0,5.160376,1540298755,54,1
2,1540298755_54_2,3208.0,-1.712692,1540298755,54,2
3,1540298755_54_3,2472.0,0.013136,1540298755,54,3
4,1540298755_54_4,2456.0,0.033973,1540298755,54,4


In [32]:
def add_labels_to_cnc_df(df, df_labels, col_list_case=None):

    df_labels["unix_date"] = df_labels["unix_date"].astype(int)

    if col_list_case is not None:
        df = df.merge(df_labels[["unix_date"] + col_list_case], on=["unix_date"])

    # select all rows in df_labels where "failed_tools" is not empty
    df_labels = df_labels[df_labels["failed_tools"].notna()].copy()

    # convert each "failed_tools" string to a list
    df_labels["failed_tools"] = df_labels["failed_tools"].apply(lambda x: x.split(" "))

    df_labels = df_labels.explode('failed_tools')

    # change dtype of "failed" column to int
    df_labels = df_labels[["unix_date", "failed", "failed_tools"]]
    df_labels["failed"] = df_labels["failed"].astype(int)

    # drop any rows where "failed_tools" is not a numeric value
    df_labels = df_labels[df_labels["failed_tools"].apply(lambda x: x.isnumeric())]
    df_labels["failed_tools"] = df_labels["failed_tools"].astype(int)

    df = pd.merge(df, df_labels[["unix_date", "failed", "failed_tools"]], left_on=["unix_date", "tool_no"], right_on=["unix_date", "failed_tools"], how="left").drop(columns=["failed_tools"])
    df["failed"] = df["failed"].fillna(0).astype(int)

    # drop "failed_tools" column
    # dfm = dfm.drop(columns=["failed_tools"])

    df["y"] = df["failed"].astype(int)
    df = df.drop(columns=["failed"])

    return df

In [33]:
df_new = add_labels_to_cnc_df(df, df_labels, col_list_case=['case_tool_54'])
df_new.head()

,id,current__length,current__kurtosis,unix_date,tool_no,index_no,case_tool_54,y
0,1540298755_54_0,2624.0,-1.802253,1540298755,54,0,1,1
1,1540298755_54_1,8504.0,5.160376,1540298755,54,1,1,1
2,1540298755_54_2,3208.0,-1.712692,1540298755,54,2,1,1
3,1540298755_54_3,2472.0,0.013136,1540298755,54,3,1,1
4,1540298755_54_4,2456.0,0.033973,1540298755,54,4,1,1


In [28]:
# save to csv
df_new.to_csv("cnc_features_54_with_labels.csv", index=False)